In [2]:
from lusidtools.jupyter_tools import toggle_code

"""Bitcoin Futures Carry Trade

Attributes
----------
futures
crypto
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Carry Trade with CME Bitcoin futures and spot BTC

## Table of Contents
* [1. Create Portfolio](#-Create-Portfolio)
* [2. Create Instruments](#-Create-Instruments)
* [3. Quotes](#-Quotes)
* [4. Valuations](#-Valuations)
* [5. Transactions](#-Transactions)

In [3]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
from utilities.instrument_utils import create_property, add_utc_to_df, valuation_response_to_df

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Create API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), app_name="LusidJupyterNotebook"
)

# 1. Introduction

We begin by looking at booking a carry trade using a spot and future instrument, and cover life cycle and valuation in LUSID.

In [4]:
# Initiate required LUSID APIs
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

## 1.1 Create Instruments

We begin by creating the 2 instruments in LUSID that will form part of the trading strategy:

- Bitcoin CME Future - Aug 2022
- Bitcoin Spot

We imagine taking a position in both bitcoin spot and the future to take advantage of the spread. This could also be achieved using a Bitcoin ETF.


In [5]:
# Define function that creates futures
def create_futures_contract(
        name,
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        country_id,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        identifier,
        convention="Invalid",
        properties: list = None,
) -> lm.InstrumentDefinition:
    contract_details = lm.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        country=country_id,
        description=name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
        convention=convention,
    )
    future = lm.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=contract_details,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=lm.ExoticInstrument(
            instrument_format=lm.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # Persist the instrument and upsert
    return lm.InstrumentDefinition(
        name=name,
        identifiers={
            "ClientInternal": lm.InstrumentIdValue(
                identifier
            )
        },
        definition=future,
        properties=properties
    )

def create_currency_instrument(
        name,
        currency_id,
        properties: list = None,
) -> lm.InstrumentDefinition:

    return lm.InstrumentDefinition(
        name=name,
        identifiers={
            "Currency": lm.InstrumentIdValue(
                currency_id
            )
        },
        properties=properties,
    )

def populate_property_values(
        property_scope: str,
        property_code: str,
        value: float,
        unit: str = None,
        property_display_name: str = None
):
    return lm.PerpetualProperty(
        key=f"Instrument/{property_scope}/{property_code}",
        value=lm.PropertyValue(
            metric_value=lm.MetricValue(
                value=value,
                unit=unit
            )
        ),
    )

# Define a function to upsert instrument
def create_request_body(*args: lm.InstrumentDefinition) -> dict:
    instrument_definitions = args
    return {
        k:v for k,v in enumerate(instrument_definitions)
    }

In [6]:
# Set scope for instruments and properties
scope = "CarryDemo"

In [7]:
# Details for margin requirements
margins_dict = {
    "InitialMargin" : 33827,
    "SecondaryMargin" : 30752
}

for key in margins_dict.keys():
    try:
        property_definition_request = lm.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code=key,
            display_name=key,
            data_type_id=lm.ResourceId(
                scope="system",
                code="number",
            ),
            life_time="Perpetual",
        )

        property_definitions_api.create_property_definition(
            create_property_definition_request=property_definition_request
        )
        print(f"Property with code: {key} has been created.")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/CarryDemo/InitialMargin' because it already exists.
Error creating Property Definition 'Instrument/CarryDemo/SecondaryMargin' because it already exists.


In [8]:
# Create list of instrument properties
margin_properties = [
    populate_property_values(
        property_scope=scope,
        property_code=key,
        value=value,
    )
    for key, value in margins_dict.items()
]

# Create 'Future' instrument in LUSID
fut_id = "BTCQ2"
future_definition = create_futures_contract(
    name="CME Bitcoin Fut Aug22",
    dom_ccy="USD",
    contract_code="BTC",
    contract_month="Q",
    contract_size="5",
    country_id="US",
    exchange_code="CME",
    exchange_name="Chicago Mercantile Exchange",
    ticker_step="5",
    unit_value="25",
    ref_spot_price=None,
    start_date=datetime(2022, 2, 8, tzinfo=pytz.utc),
    maturity_date=datetime(2022, 8, 26, tzinfo=pytz.utc),
    identifier=fut_id,
    properties=margin_properties
)

# Create Bitcoin currency instrument - notice use of XBT to meet ISO standards
spot_id = "XBT"
spot_definition = create_currency_instrument(
    name="Bitcoin",
    currency_id="XBT",
)

In [9]:
request_body = create_request_body(future_definition, spot_definition)
response = instruments_api.upsert_instruments(request_body=request_body)

if len(response.failed) == 0:
    print(f"{len(response.values)} instruments upserted to LUSID")
else:
    print(response.failed)

2 instruments upserted to LUSID


# 2.0 Portfolio Setup

In [10]:
# Create sub-holding key property
property_code = "SecurityType"
sec_type_property_key = f"Transaction/{scope}/{property_code}"

create_property(api_factory, "Transaction", scope, property_code, property_code, "string")

Property Transaction/CarryDemo/SecurityType already exists


In [11]:
# Set portfolio details and upsert
portfolio_code = "CarryTradeWithFuture"
base_ccy = "USD"

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency=base_ccy,
            created="2010-01-01",
            sub_holding_keys=[sec_type_property_key],
            instrument_scopes=[scope]
        ),
    )
    print(f"Portfolio with following resource IDs: scope: {scope} | code: {portfolio_code}")
except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'CarryTradeWithFuture' because it already exists in scope 'CarryDemo'.


### 3.0 Transaction Configuration and Initial Trades

In [12]:
transaction_group = "CarryDemo"

new_transaction_config = [
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="OpenShortContract",
                description="A futures transaction type, opening a position with allocated margin.",
                transaction_class="default",
                transaction_group=transaction_group,
                transaction_roles="AllRoles",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                name="Stock",
                direction=-1,
                properties=None,
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/CarryDemo/SecurityType",
                        map_from=sec_type_property_key,
                    )
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                name="Collateral",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=sec_type_property_key,
                        set_to="Collateral",
                    )
                ],
            )
        ],
        properties=None,
    ),
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="VirtualCurrencyBuy",
                description="A virtual currency buy transaction.",
                transaction_class="default",
                transaction_group=transaction_group,
                transaction_roles="AllRoles",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                name="VirtualCurrency",
                direction=1,
                properties=None,
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=f"Transaction/CarryDemo/SecurityType",
                        map_from=sec_type_property_key,
                    )
                ],
            ),
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                name="Cash",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=sec_type_property_key,
                        set_to="Cash",
                    )
                ],
            )
        ],
        properties=None,
    ),
    lm.TransactionConfigurationDataRequest(
        aliases=[
            lm.TransactionConfigurationTypeAlias(
                type="MarginCall",
                description="Increase cost basis and adjust collateral.",
                transaction_class="default",
                transaction_group=transaction_group,
                transaction_roles="AllRoles",
            )
        ],
        movements=[
            lm.TransactionConfigurationMovementDataRequest(
                movement_types="VariationMargin",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[],
            ),lm.TransactionConfigurationMovementDataRequest(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[
                    lm.TransactionPropertyMappingRequest(
                        property_key=sec_type_property_key,
                        set_to="Collateral",
                    )
                ],
            )
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

new_txn_config

{'links': [{'description': None,
            'href': 'https://swagpotato.lusid.com/api/api/systemconfiguration/transactions',
            'method': 'POST',
            'relation': 'Transactions'},
           {'description': None,
            'href': 'https://swagpotato.lusid.com/api/api/schemas/entities/TransactionSetConfigurationData',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://swagpotato.lusid.com/api/',
            'method': 'GET',
            'relation': 'PropertySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://swagpotato.lusid.com/app/insights/logs/0HMKMTRNB57N6:0000003C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'side_definitions': [{'amount': 'Txn:TotalConsideration',
                       'currency': 'Txn:SettlementCurrency',
         

In [13]:
# Set trade details
trade_date = datetime(2022, 2, 8, tzinfo=pytz.utc)
trade_ccy = "USD"
fut_units = 100
initial_margin = 33827
fut_price = 44118.44531
spot_price = 44340
spot_units = 500
spot_cost = spot_units * spot_price

# Create trade request
open_position = [
    lm.TransactionRequest(
        transaction_id="TXN001",
        type="OpenShortContract",
        instrument_identifiers={
            "Instrument/default/ClientInternal": f"{fut_id}"
        },
        transaction_date=trade_date,
        settlement_date=trade_date,
        units=fut_units,
        transaction_price=lm.TransactionPrice(price=fut_price, type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=initial_margin, currency=trade_ccy),
        exchange_rate=1,
        transaction_currency=trade_ccy,
        source=transaction_group,
        properties={
            sec_type_property_key: lm.PerpetualProperty(
                key=sec_type_property_key,
                value=lm.PropertyValue(
                    label_value="Future"
                )
            )
        }
    ),
    lm.TransactionRequest(
        transaction_id="TXN002",
        type="VirtualCurrencyBuy",
        instrument_identifiers={
            "Instrument/default/LusidInstrumentId": f"CCY_{spot_id}"
        },
        transaction_date=trade_date,
        settlement_date=trade_date,
        units=spot_units,
        transaction_price=lm.TransactionPrice(price=spot_price, type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=spot_cost, currency=trade_ccy),
        exchange_rate=spot_price,
        transaction_currency="XBT",
        source=transaction_group,
        properties={
            sec_type_property_key: lm.PerpetualProperty(
                key=sec_type_property_key,
                value=lm.PropertyValue(
                    label_value="VirtualCurrency"
                )
            )
        }
    )

]

response = transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=open_position
)

print(f"Transaction updated for effective time: {response.version.effective_from}.")

Transaction updated for effective time: 2022-09-15 10:41:12.482564+00:00.


# 4.0 Market Data & Configuration Recipe

In [14]:
quotes_df = pd.read_csv("data/bitcoin_market_data.csv")
quotes_df["Price Date"] = quotes_df["Price Date"].apply(lambda x: parse(x, dayfirst=True))
add_utc_to_df(quotes_df)
quotes_df.head()

,Price Date,Close,Identifier,Identifier Type,Units
0,2022-08-15 00:00:00+00:00,"24,240.00",XBT/USD,CurrencyPair,XBT/USD
1,2022-08-14 00:00:00+00:00,"24,315.00",XBT/USD,CurrencyPair,XBT/USD
2,2022-08-12 00:00:00+00:00,"24,240.00",XBT/USD,CurrencyPair,XBT/USD
3,2022-08-11 00:00:00+00:00,"24,245.00",XBT/USD,CurrencyPair,XBT/USD
4,2022-08-10 00:00:00+00:00,"23,655.00",XBT/USD,CurrencyPair,XBT/USD


In [15]:
# Set the market data scope
market_data_scope = "BitcoinFutures"

# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["Identifier"],
                    instrument_id_type=row["Identifier Type"],
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["Price Date"],
            ),
            metric_value=lm.MetricValue(value=row["Close"], unit=row["Units"]),
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 334 quotes loaded.


In [16]:
# Set recipe code
recipe_scope = "Finbourne-Examples"
recipe_code = "BitcoinFutures"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D",
            ),
            lm.MarketDataKeyRule(
                key="Fx.CurrencyPair.*",
                supplier="Lusid",
                data_scope=market_data_scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D",
            ),
        ],
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Future",
                parameters="{}",
            )
        ],
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)

print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2022-09-16 13:37:58.974314+00:00.


In [17]:
def margin_work_flow(
        api_factory: lusid.utilities.ApiClientFactory,
        portfolio_scope: str,
        portfolio_code: str,
        recipe_scope: str,
        recipe_code: str,
        effective_date: datetime=datetime,
        # initial_margin_property_key: str,
        # secondary_margin_property_key: str,
        # margin_call_transaction_type: str,
        # instrument_types: list=["Future"],
        group_by: list=["Instrument/default/LusidInstrumentId"]
) -> None:

    metrics = [
        lm.AggregateSpec(
            key="Valuation/PvInPortfolioCcy",
            op="Value"
        ),
        lm.AggregateSpec(
            key="Valuation/PnL/Unrealised",
            op="Value"
        ),
        lm.AggregateSpec(
            key="Instrument/InstrumentType",
            op="Value"
        ),
        lm.AggregateSpec(
            key=sec_type_property_key,
            op="Value"
        ),
    ]

    valuation_results = api_factory.build(lusid.api.AggregationApi).get_valuation(
        valuation_request=lm.ValuationRequest(
            recipe_id=lm.ResourceId(scope=recipe_scope, code=recipe_code),
            metrics=metrics,
            group_by=group_by,
            portfolio_entity_ids=[
                lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=effective_date),

        )
    )

    # valuation_results = lm.ValuationRequest(
    #     recipe_id=lm.ResourceId(scope=recipe_scope, code=recipe_code),
    #     metrics=metrics,
    #     group_by=group_by,
    #     portfolio_entity_ids=[
    #         lm.PortfolioEntityId(scope=portfolio_scope, code=portfolio_code)
    # ],
    # valuation_schedule=lm.ValuationSchedule(effective_at=effective_date),
    #
    # )

    return valuation_results

    #TODO: filter to list of included instrument types for MtM margin workflow
    # for instrument in instrument_types:
    #     if instrument_type in valuation_results.values["Instrument/InstrumentType"]:
    #         #TODO: do some margin stuff

In [26]:
effective_date = datetime(2022, 6, 10, tzinfo=pytz.utc)

results = margin_work_flow(
    api_factory=api_factory,
    portfolio_scope=scope,
    portfolio_code=portfolio_code,
    recipe_scope=recipe_scope,
    recipe_code=recipe_code,
    effective_date=effective_date,
    #group_by=["Instrument/default/LusidInstrumentId", sec_type_property_key]

)

In [27]:
val_df = valuation_response_to_df(results)
val_df

,Valuation/PvInPortfolioCcy,Instrument/InstrumentType,Transaction/CarryDemo/SecurityType,Valuation/PnL/Unrealised
0,"-14,541,902.34",Future,Future,"-14,508,075.34"
1,nan,CashPerpetual,None,0.00
2,"14,460,000.00",CashPerpetual,VirtualCurrency,0.00


In [28]:
val_df["Valuation/PvInPortfolioCcy"].sum()

-81902.34499999881

In [29]:
results.data

[{'Valuation/PvInPortfolioCcy': -14541902.344999999,
  'Valuation/PnL/Unrealised': -14508075.344999999,
  'Instrument/InstrumentType': 'Future',
  'Transaction/CarryDemo/SecurityType': 'Future'},
 {'Valuation/PvInPortfolioCcy': None,
  'Valuation/PnL/Unrealised': 0.0,
  'Instrument/InstrumentType': 'CashPerpetual',
  'Transaction/CarryDemo/SecurityType': None},
 {'Valuation/PvInPortfolioCcy': 14460000.0,
  'Valuation/PnL/Unrealised': 0.0,
  'Instrument/InstrumentType': 'CashPerpetual',
  'Transaction/CarryDemo/SecurityType': 'VirtualCurrency'}]